In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


In [2]:
from openai import OpenAI
from google.colab import userdata
api_key = userdata.get('api_key')

client = OpenAI(api_key=api_key)


instruct="You are a recipe search engine. The user will send you a list of ingredients, and you will search your database for recipes that can be cooked with just those ingredients without requiring any extra ingredients. Search the Recipe Database every time, do not invent a recipe. The recipe must contain NO EXTRA INGREDIENTS, only the ingredients that the user has. The user should have to go buy any new recipes. Your output should state the recipe name, ingredients (exactly as they appear in the dataset, including measurements), and instructions. Your output should contain no other text. Only use recipes that exist in the database. Remember: Every ingredient in the recipe, the user must have. Not every ingredient the user has must be in the recipe though"

assistant = client.beta.assistants.create(
  name="Recipe Search",
  instructions=instruct,
  model="gpt-4-turbo",
  tools=[{"type": "file_search"}],
)

In [3]:
# RECIPE DATABASE / VECTOR STORE UPLOAD


vector_store = client.beta.vector_stores.create(name="RecipeDB")

file_paths = ["/content/drive/MyDrive/AssistantsAPI/recipes_1_of_6.json", "/content/drive/MyDrive/AssistantsAPI/recipes_2_of_6.json", "/content/drive/MyDrive/AssistantsAPI/recipes_4_of_6.json", "/content/drive/MyDrive/AssistantsAPI/recipes_5_of_6.json", "/content/drive/MyDrive/AssistantsAPI/recipes_6_of_6.json"]
file_streams = [open(path, "rb") for path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

# print status
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=5, failed=0, in_progress=0, total=5)


In [4]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [9]:

thread = client.beta.threads.create()

while True:
    user_input = input("\nUser > ")

    message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content=user_input
    )

    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions=instruct
    )

    print("Searching Recipe Database...")
    while True:
        run_status = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
        if run_status.status == 'completed':
            break
        elif run_status.status == 'failed':
            print("Run failed:", run_status.error)
            break

    messages = list(client.beta.threads.messages.list(thread_id=thread.id))
    message_content = messages[0].content[0].text.value



    print("\n\n" + message_content)


User > Garlic, ginger, pasta, rice, noodles, milk, chocolate
Searching Recipe Database...


**Cold Spicy Noodles**  
- Ingredients:
  - Noodles
  - Garlic
  - Ginger
  - Sesame oil
  - Soy sauce
  - Chile oil
  - Bean sauce
  - Sugar
  - Green onions
- Instructions:
  - Cook noodles in a large pot of boiling salted water until just tender, then drain in a colander. Rinse with cold water until cool. Toss in a large bowl with sesame oil. In a small bowl, stir together soy sauce, chile oil, bean sauce, ginger, garlic, and sugar. Pour over noodles, add green onions, and toss well.
  - If taking on a picnic, transfer noodles to a container for transport. To assemble, put noodles in serving bowls and offer with desired toppings and chopsticks【4:0†source】.


KeyboardInterrupt: Interrupted by user